<a href="https://colab.research.google.com/github/yarlagaddaprashanth/Zeotap/blob/main/rules_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class Node:
    def __init__(self, node_type, left=None, right=None, value=None):
        self.node_type = node_type  # "operator" for AND/OR, "operand" for conditions
        self.left = left  # left child node
        self.right = right  # right child node
        self.value = value  # value for operand nodes (e.g., comparison values)


In [ ]:
def create_rule(rule_string):
    # Simplified parsing (would ideally use a more robust parser or an existing library)
    tokens = rule_string.split(" ")

    # Recursively create an AST for conditions
    if "AND" in tokens or "OR" in tokens:
        operator_index = tokens.index("AND") if "AND" in tokens else tokens.index("OR")
        left_rule = " ".join(tokens[:operator_index])
        right_rule = " ".join(tokens[operator_index+1:])
        node_type = "operator"
        operator = tokens[operator_index]
        return Node(node_type=node_type, left=create_rule(left_rule), right=create_rule(right_rule), value=operator)

    # Operand node (leaf node)
    condition = rule_string.strip()
    return Node(node_type="operand", value=condition)


In [ ]:
def combine_rules(rules):
    combined = None
    for rule in rules:
        if combined is None:
            combined = create_rule(rule)
        else:
            # Combine rules using OR (as an example)
            combined = Node(node_type="operator", left=combined, right=create_rule(rule), value="OR")
    return combined


In [ ]:
def evaluate_rule(ast, data):
    if ast.node_type == "operand":
        # Evaluate condition, e.g., "age > 30"
        condition = ast.value.split(" ")
        attribute, operator, value = condition[0], condition[1], condition[2]
        if operator == ">":
            return data[attribute] > int(value)
        elif operator == "<":
            return data[attribute] < int(value)
        elif operator == "=":
            return data[attribute] == value.strip("'")

    if ast.node_type == "operator":
        if ast.value == "AND":
            return evaluate_rule(ast.left, data) and evaluate_rule(ast.right, data)
        elif ast.value == "OR":
            return evaluate_rule(ast.left, data) or evaluate_rule(ast.right, data)

# Example usage
rule1 = "age > 30 AND department = 'Sales'"
rule2 = "age < 25 AND department = 'Marketing'"
combined_ast = combine_rules([rule1, rule2])

data = {"age": 35, "department": "Sales", "salary": 60000, "experience": 3}
result = evaluate_rule(combined_ast, data)
print(result)  # Outputs True or False based on the rule


True
